In [323]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from fuzzywuzzy import fuzz
import re

import psycopg2
from psycopg2 import Error
import csv

## Importing the unstructured json data - Users, Brands, and Receipts

In [263]:
users_json = pd.read_json('users 2.json', lines=True)
brands_json = pd.read_json('brands (1).json', lines=True)
receipts_json = pd.read_json('receipts (1).json', lines=True)

In [264]:
receipts_json

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [265]:
users_json

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


## Preprocessing the unstructured json data to structured format - to be used to write SQL queries on

In [266]:
# Transform the _id column to keep only the $oid value
users_json['_id'] = users_json['_id'].apply(lambda x: x['$oid'])


In [267]:
users_json['createdDate'] = users_json['createdDate'].apply(lambda x: x['$date'])


In [268]:
users_json['lastLogin'] = users_json['lastLogin'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else np.nan)

In [269]:
users_json

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN


In [270]:
# Convert createdDate to ISO 8601 format
users_json['createdDate'] = pd.to_datetime(users_json['createdDate'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

# Convert lastLogin to ISO 8601 format
users_json['lastLogin'] = pd.to_datetime(users_json['lastLogin'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

In [273]:
users_json

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03T15:25:30.554000Z,2021-01-03T15:25:30.596999Z,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN


In [274]:
receipts_json['rewardsReceiptItemList']

0       [{'barcode': '4011', 'description': 'ITEM NOT ...
1       [{'barcode': '4011', 'description': 'ITEM NOT ...
2       [{'needsFetchReview': False, 'partnerItemId': ...
3       [{'barcode': '4011', 'description': 'ITEM NOT ...
4       [{'barcode': '4011', 'description': 'ITEM NOT ...
                              ...                        
1114    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1115                                                  NaN
1116                                                  NaN
1117    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1118                                                  NaN
Name: rewardsReceiptItemList, Length: 1119, dtype: object

In [275]:
# Explode the rewardsReceiptItemList to create individual rows for each item
receipts_exploded = receipts_json.explode('rewardsReceiptItemList').reset_index(drop=True)

# Convert the dictionaries in the exploded column to separate columns
items_df = pd.json_normalize(receipts_exploded['rewardsReceiptItemList'])

# Drop the original rewardsReceiptItemList column
df_exploded = receipts_exploded.drop(columns=['rewardsReceiptItemList'])

# Concatenate the exploded DataFrame with the items DataFrame
result_df = pd.concat([df_exploded, items_df], axis=1)

# Display the resulting DataFrame
print(result_df)

                                       _id  bonusPointsEarned  \
0     {'$oid': '5ff1e1eb0a720f0523000575'}              500.0   
1     {'$oid': '5ff1e1bb0a720f052300056b'}              150.0   
2     {'$oid': '5ff1e1bb0a720f052300056b'}              150.0   
3     {'$oid': '5ff1e1f10a720f052300057a'}                5.0   
4     {'$oid': '5ff1e1ee0a7214ada100056f'}                5.0   
...                                    ...                ...   
7376  {'$oid': '603d0b710a720fde1000042a'}                NaN   
7377  {'$oid': '603cf5290a720fde10000413'}                NaN   
7378  {'$oid': '603ce7100a7217c72c000405'}               25.0   
7379  {'$oid': '603ce7100a7217c72c000405'}               25.0   
7380  {'$oid': '603c4fea0a7217c72c000389'}                NaN   

                                bonusPointsEarnedReason  \
0     Receipt number 2 completed, bonus point schedu...   
1     Receipt number 5 completed, bonus point schedu...   
2     Receipt number 5 completed, bonus po

In [280]:
result_df['_id'].value_counts()

_id
{'$oid': '600f2fc80a720f0535000030'}    459
{'$oid': '600f39c30a7214ada2000030'}    450
{'$oid': '600f24970a720f053500002f'}    381
{'$oid': '600f0cc70a720f053500002c'}    217
{'$oid': '600a1a8d0a7214ada2000008'}    203
                                       ... 
{'$oid': '6010bdfa0a7214ada200005a'}      1
{'$oid': '6010be200a7214ada200005e'}      1
{'$oid': '60118c0f0a7214ada200007e'}      1
{'$oid': '60118bea0a720f0535000064'}      1
{'$oid': '603c4fea0a7217c72c000389'}      1
Name: count, Length: 1119, dtype: int64

In [281]:
result_df.to_csv('results.csv')

In [282]:
receipts_df['_id'].value_counts()

_id
5ff1e1eb0a720f0523000575    1
601b48df0a720f05f4000238    1
601c11410a720f05f400028e    1
601c2d630a7214ad280002a7    1
601be7120a720f05f400027a    1
                           ..
60088a670a720f05fa0000e0    1
600788cd0a7214ad8900003a    1
60085bad0a720f05fa00006c    1
60088e670a720f05fa00011d    1
603c4fea0a7217c72c000389    1
Name: count, Length: 1119, dtype: int64

In [283]:
receipts_df.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId', 'barcode',
       'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'brand_name'],
      dtype='object')

In [285]:
result_df.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId', 'barcode',
       'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMeta

In [287]:
receipts_drop_columns = ['barcode',
       'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity']

receipts_df.drop(columns=receipts_drop_columns, axis=1, inplace=True)

In [288]:
receipts_df

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,brand_name
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03T15:25:31.000000Z,2021-01-03T15:25:31.000000Z,2021-01-03T15:25:31.000000Z,2021-01-03T15:25:36.000000Z,2021-01-03T15:25:31.000000Z,500.0,2021-01-03T00:00:00.000000Z,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03T15:24:43.000000Z,2021-01-03T15:24:43.000000Z,2021-01-03T15:24:43.000000Z,2021-01-03T15:24:48.000000Z,2021-01-03T15:24:43.000000Z,150.0,2021-01-02T15:24:43.000000Z,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03T15:25:37.000000Z,2021-01-03T15:25:37.000000Z,NaN,2021-01-03T15:25:42.000000Z,NaN,5.0,2021-01-03T00:00:00.000000Z,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03T15:25:34.000000Z,2021-01-03T15:25:34.000000Z,2021-01-03T15:25:34.000000Z,2021-01-03T15:25:39.000000Z,2021-01-03T15:25:34.000000Z,5.0,2021-01-03T00:00:00.000000Z,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03T15:25:06.000000Z,2021-01-03T15:25:06.000000Z,2021-01-03T15:25:11.000000Z,2021-01-03T15:25:11.000000Z,2021-01-03T15:25:06.000000Z,5.0,2021-01-02T15:25:06.000000Z,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01T10:22:27.000000Z,2021-03-01T10:22:27.000000Z,NaN,2021-03-01T10:22:28.000000Z,NaN,25.0,2020-08-17T00:00:00.000000Z,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33,Nutri-Grain
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01T15:42:41.873000Z,2021-03-01T15:42:41.873000Z,NaN,2021-03-01T15:42:41.873000Z,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01T14:07:37.664000Z,2021-03-01T14:07:37.664000Z,NaN,2021-03-01T14:07:37.664000Z,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01T13:07:28.000000Z,2021-03-01T13:07:28.000000Z,NaN,2021-03-01T13:07:29.000000Z,NaN,25.0,2020-08-17T00:00:00.000000Z,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33,Nutri-Grain


In [289]:
results_drop_cols = ['bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId']

result_df.drop(columns=results_drop_cols, axis=1, inplace=True)

In [290]:
result_df

,_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,originalReceiptItemText,itemNumber,originalMetaBriteQuantityPurchased,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,{'$oid': '5ff1e1eb0a720f0523000575'},4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5ff1e1bb0a720f052300056b'},4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '5ff1e1bb0a720f052300056b'},028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '5ff1e1f10a720f052300057a'},NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5ff1e1ee0a7214ada100056f'},4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,{'$oid': '603ce7100a7217c72c000405'},B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,mueller austria hypergrind precision electric ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,{'$oid': '603ce7100a7217c72c000405'},B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [69]:
# Function to extract keys from nested dictionaries
def extract_key(row, key):
    if isinstance(row, list) and len(row) > 0:
        return row[0].get(key)
    else:
        return None

# Extract specific keys from the nested dictionaries
keys_to_extract = ['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview', 'partnerItemId',
                   'preventTargetGapPoints', 'quantityPurchased', 'userFlaggedBarcode', 'userFlaggedNewItem',
                   'userFlaggedPrice', 'userFlaggedQuantity']

In [70]:
for key in keys_to_extract:
    receipts_json[key] = receipts_json['rewardsReceiptItemList'].apply(lambda x: extract_key(x, key))

In [71]:
# Drop the original rewardsReceiptsItemList column
receipts_json = receipts_json.drop(columns=['rewardsReceiptItemList'])

# Display the transformed DataFrame
print(receipts_json)

                                       _id  bonusPointsEarned  \
0     {'$oid': '5ff1e1eb0a720f0523000575'}              500.0   
1     {'$oid': '5ff1e1bb0a720f052300056b'}              150.0   
2     {'$oid': '5ff1e1f10a720f052300057a'}                5.0   
3     {'$oid': '5ff1e1ee0a7214ada100056f'}                5.0   
4     {'$oid': '5ff1e1d20a7214ada1000561'}                5.0   
...                                    ...                ...   
1114  {'$oid': '603cc0630a720fde100003e6'}               25.0   
1115  {'$oid': '603d0b710a720fde1000042a'}                NaN   
1116  {'$oid': '603cf5290a720fde10000413'}                NaN   
1117  {'$oid': '603ce7100a7217c72c000405'}               25.0   
1118  {'$oid': '603c4fea0a7217c72c000389'}                NaN   

                                bonusPointsEarnedReason  \
0     Receipt number 2 completed, bonus point schedu...   
1     Receipt number 5 completed, bonus point schedu...   
2                            All-receipts 

In [73]:
receipts_json['barcode']

0             4011
1             4011
2             None
3             4011
4             4011
           ...    
1114    B076FJ92M4
1115          None
1116          None
1117    B076FJ92M4
1118          None
Name: barcode, Length: 1119, dtype: object

In [75]:
receipts_json.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId', 'barcode',
       'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity'],
      dtype='object')

In [79]:
receipts_json

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},...,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},...,1,1,None,1,None,1.0,None,None,None,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},...,None,None,False,1,True,NaN,4011,True,26.00,3.0
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},...,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},...,1,1,None,1,None,1.0,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},...,22.97,22.97,None,0,None,1.0,None,None,None,NaN
1115,603d0b710a720fde1000042a,NaN,NaN,1614613361873,1614613361873,NaN,{'$date': 1614613361873},NaN,NaN,NaN,...,None,None,None,None,None,NaN,None,None,None,NaN
1116,603cf5290a720fde10000413,NaN,NaN,1614607657664,1614607657664,NaN,{'$date': 1614607657664},NaN,NaN,NaN,...,None,None,None,None,None,NaN,None,None,None,NaN
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},...,22.97,22.97,None,0,None,1.0,None,None,None,NaN


In [83]:
# Transform the _id column to keep only the $oid value
receipts_json['_id'] = receipts_json['_id'].apply(lambda x: x['$oid'])
receipts_json['createDate'] = receipts_json['createDate'].apply(lambda x: x['$date'])
receipts_json['dateScanned'] = receipts_json['dateScanned'].apply(lambda x: x['$date'])
receipts_json['modifyDate'] = receipts_json['modifyDate'].apply(lambda x: x['$date'])

In [85]:
receipts_json['finishedDate'] = receipts_json['finishedDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else np.nan)

receipts_json['pointsAwardedDate'] = receipts_json['pointsAwardedDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else np.nan)
receipts_json['purchaseDate'] = receipts_json['purchaseDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else np.nan)

In [292]:
result_df.columns

Index(['_id', 'barcode', 'description', 'finalPrice', 'itemPrice',
       'needsFetchReview', 'partnerItemId', 'preventTargetGapPoints',
       'quantityPurchased', 'userFlaggedBarcode', 'userFlaggedNewItem',
       'userFlaggedPrice', 'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

In [293]:
result_df['_id'] = result_df['_id'].apply(lambda x: x['$oid'])

In [295]:
result_df.to_csv('receipts_item.csv')

In [296]:
#receipts_df.to_csv('receipts.csv')

In [297]:
result_df.columns

Index(['_id', 'barcode', 'description', 'finalPrice', 'itemPrice',
       'needsFetchReview', 'partnerItemId', 'preventTargetGapPoints',
       'quantityPurchased', 'userFlaggedBarcode', 'userFlaggedNewItem',
       'userFlaggedPrice', 'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

In [89]:
receipts_json

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,...,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,...,1,1,None,1,None,1.0,None,None,None,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,...,None,None,False,1,True,NaN,4011,True,26.00,3.0
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,...,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,...,1,1,None,1,None,1.0,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,NaN,1614594148000,NaN,25.0,1.597622e+12,...,22.97,22.97,None,0,None,1.0,None,None,None,NaN
1115,603d0b710a720fde1000042a,NaN,NaN,1614613361873,1614613361873,NaN,1614613361873,NaN,NaN,NaN,...,None,None,None,None,None,NaN,None,None,None,NaN
1116,603cf5290a720fde10000413,NaN,NaN,1614607657664,1614607657664,NaN,1614607657664,NaN,NaN,NaN,...,None,None,None,None,None,NaN,None,None,None,NaN
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,NaN,1614604049000,NaN,25.0,1.597622e+12,...,22.97,22.97,None,0,None,1.0,None,None,None,NaN


In [90]:
# Convert createdDate to ISO 8601 format
receipts_json['createDate'] = pd.to_datetime(receipts_json['createDate'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

# Convert lastLogin to ISO 8601 format
receipts_json['dateScanned'] = pd.to_datetime(receipts_json['dateScanned'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
receipts_json['modifyDate'] = pd.to_datetime(receipts_json['modifyDate'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
receipts_json['finishedDate'] = pd.to_datetime(receipts_json['finishedDate'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
receipts_json['pointsAwardedDate'] = pd.to_datetime(receipts_json['pointsAwardedDate'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

receipts_json['purchaseDate'] = pd.to_datetime(receipts_json['purchaseDate'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')


In [91]:
receipts_json

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03T15:25:31.000000Z,2021-01-03T15:25:31.000000Z,2021-01-03T15:25:31.000000Z,2021-01-03T15:25:36.000000Z,2021-01-03T15:25:31.000000Z,500.0,2021-01-03T00:00:00.000000Z,...,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03T15:24:43.000000Z,2021-01-03T15:24:43.000000Z,2021-01-03T15:24:43.000000Z,2021-01-03T15:24:48.000000Z,2021-01-03T15:24:43.000000Z,150.0,2021-01-02T15:24:43.000000Z,...,1,1,None,1,None,1.0,None,None,None,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03T15:25:37.000000Z,2021-01-03T15:25:37.000000Z,NaN,2021-01-03T15:25:42.000000Z,NaN,5.0,2021-01-03T00:00:00.000000Z,...,None,None,False,1,True,NaN,4011,True,26.00,3.0
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03T15:25:34.000000Z,2021-01-03T15:25:34.000000Z,2021-01-03T15:25:34.000000Z,2021-01-03T15:25:39.000000Z,2021-01-03T15:25:34.000000Z,5.0,2021-01-03T00:00:00.000000Z,...,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03T15:25:06.000000Z,2021-01-03T15:25:06.000000Z,2021-01-03T15:25:11.000000Z,2021-01-03T15:25:11.000000Z,2021-01-03T15:25:06.000000Z,5.0,2021-01-02T15:25:06.000000Z,...,1,1,None,1,None,1.0,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01T10:22:27.000000Z,2021-03-01T10:22:27.000000Z,NaN,2021-03-01T10:22:28.000000Z,NaN,25.0,2020-08-17T00:00:00.000000Z,...,22.97,22.97,None,0,None,1.0,None,None,None,NaN
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01T15:42:41.873000Z,2021-03-01T15:42:41.873000Z,NaN,2021-03-01T15:42:41.873000Z,NaN,NaN,NaN,...,None,None,None,None,None,NaN,None,None,None,NaN
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01T14:07:37.664000Z,2021-03-01T14:07:37.664000Z,NaN,2021-03-01T14:07:37.664000Z,NaN,NaN,NaN,...,None,None,None,None,None,NaN,None,None,None,NaN
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01T13:07:28.000000Z,2021-03-01T13:07:28.000000Z,NaN,2021-03-01T13:07:29.000000Z,NaN,25.0,2020-08-17T00:00:00.000000Z,...,22.97,22.97,None,0,None,1.0,None,None,None,NaN


In [92]:
brands_json

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


In [93]:
# Function to extract values from the nested dictionaries in 'cpg' column
def extract_cpg_ref(row):
    if isinstance(row, dict) and "$ref" in row and "$id" in row["$id"]:
        return row["$ref"]
    else:
        return None

def extract_cpg_oid(row):
    if isinstance(row, dict) and "$id" in row and "$oid" in row["$id"]:
        return row["$id"]["$oid"]
    else:
        return None

# Apply the extraction functions to 'cpg' column
brands_json['cpg_ref'] = brands_json['cpg'].apply(extract_cpg_ref)
brands_json['cpg_oid'] = brands_json['cpg'].apply(extract_cpg_oid)

In [94]:
# Drop the original 'cpg' column
brands_json = brands_json.drop(columns=['cpg'])

# Display the transformed DataFrame
print(brands_json)

                                       _id       barcode            category  \
0     {'$oid': '601ac115be37ce2ead437551'}  511111019862              Baking   
1     {'$oid': '601c5460be37ce2ead43755f'}  511111519928           Beverages   
2     {'$oid': '601ac142be37ce2ead43755d'}  511111819905              Baking   
3     {'$oid': '601ac142be37ce2ead43755a'}  511111519874              Baking   
4     {'$oid': '601ac142be37ce2ead43755e'}  511111319917      Candy & Sweets   
...                                    ...           ...                 ...   
1162  {'$oid': '5f77274dbe37ce6b592e90c0'}  511111116752              Baking   
1163  {'$oid': '5dc1fca91dda2c0ad7da64ae'}  511111706328  Breakfast & Cereal   
1164  {'$oid': '5f494c6e04db711dd8fe87e7'}  511111416173      Candy & Sweets   
1165  {'$oid': '5a021611e4b00efe02b02a57'}  511111400608             Grocery   
1166  {'$oid': '6026d757be37ce6369301468'}  511111019930              Baking   

          categoryCode                 

In [98]:
brands_json

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_ref,cpg_oid
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,None,601ac114be37ce2ead437550
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,None,5332f5fbe4b03c9a25efd0ba
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,None,601ac142be37ce2ead437559
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,None,601ac142be37ce2ead437559
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,None,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,None,5f77274dbe37ce6b592e90bf
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,None,53e10d6368abd3c7065097cc
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,None,5332fa12e4b03c9a25efd1e7
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,None,5332f5f6e4b03c9a25efd0b4


In [99]:
brands_json['_id'] = brands_json['_id'].apply(lambda x: x['$oid'])

In [100]:
brands_json

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_ref,cpg_oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,None,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,None,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,None,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,None,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,None,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,None,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,None,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,None,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,None,5332f5f6e4b03c9a25efd0b4


In [322]:
#users_json.to_csv('/Users/charumangla/Downloads/fetch_assessment/users.csv')
#receipts_json.to_csv('/Users/charumangla/Downloads/fetch_assessment/receipts.csv')
#brands_json.to_csv('/Users/charumangla/Downloads/fetch_assessment/brands.csv')

## Importing all the dataframes in PostgreSql to write the SQL queries 

In [324]:
# PostgreSQL connection parameters
dbname = "charumangla"
user = "postgres"
password = "postgres"
host = "localhost"  
port = "5432"       

In [320]:
# Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Connected to PostgreSQL!")

    # Create cursor object
    cursor = conn.cursor()

    # Function to create table from CSV
    def create_table_from_csv(csv_file, table_name, columns_info):
        with open(csv_file, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            headers = next(reader)  # Get header row

            # Prepare SQL statement to create table
            create_table_sql = f"CREATE TABLE {table_name} ({', '.join(columns_info)});"
            cursor.execute(create_table_sql)
            print(f"Table {table_name} created successfully!")

            # Prepare SQL statement to copy data from CSV to table
            # The copy_sql includes the column names to ensure proper mapping
            copy_sql = f"COPY {table_name}({', '.join(headers)}) FROM stdin WITH CSV HEADER DELIMITER ','"
            f.seek(0)
            cursor.copy_expert(sql=copy_sql, file=f)
            print(f"Data imported into {table_name}!")

    # Define column information for each table (adjust as per your CSV structure)
    users_columns_info = [
        "_id VARCHAR(255)",
        "active BOOLEAN",
        "createdDate TIMESTAMP",
        "lastLogin TIMESTAMP",
        "role VARCHAR(255)",
        "signUpSource VARCHAR(255)",
        "state VARCHAR(255)"
    ]

    receipts_columns_info = [
        "_id VARCHAR(255)",
        "bonusPointsEarned INTEGER",
        "bonusPointsEarnedReason VARCHAR(255)",
        "createDate TIMESTAMP",
        "dateScanned TIMESTAMP",
        "finishedDate TIMESTAMP",
        "modifyDate TIMESTAMP",
        "pointsAwardedDate TIMESTAMP",
        "pointsEarned FLOAT",
        "purchaseDate TIMESTAMP",
        "purchasedItemCount FLOAT",
        "rewardsReceiptStatus VARCHAR(255)",
        "totalSpent FLOAT",
        "userId VARCHAR(255)",
        "brand_name VARCHAR(255)"
        # Add more columns as needed
    ]

    receipt_items_columns_info = [
        "_id VARCHAR(255)",
        "barcode VARCHAR(255)",
        "description VARCHAR(255)",
        "finalprice FLOAT",
        "itemprice FLOAT",
        "needsFetchReview VARCHAR(255)",
        "partnerItemId FLOAT",
        "preventTargetGapPoints VARCHAR(255)",
        "quantityPurchased FLOAT",
        "userFlaggedBarcode VARCHAR(255)",
        "userFlaggedNewItem VARCHAR(255)",
        "userFlaggedPrice FLOAT",
        "userFlaggedQuantity FLOAT",
        "needsFetchReviewReason VARCHAR(255)",
        "pointsNotAwardedReason VARCHAR(255)",
        "pointsPayerId VARCHAR(255)",
        "rewardsGroup VARCHAR(255)",
        "rewardsProductPartnerId VARCHAR(255)", 
        "userFlaggedDescription VARCHAR(255)",
       "originalMetaBriteBarcode VARCHAR(255)", 
       "originalMetaBriteDescription VARCHAR(255)", 
       "brandCode VARCHAR(255)",
       "competitorRewardsGroup VARCHAR(255)", 
       "discountedItemPrice FLOAT",
       "originalReceiptItemText VARCHAR(255)", 
       "itemNumber VARCHAR(255)",
       "originalMetaBriteQuantityPurchased FLOAT", 
       "targetPrice FLOAT",
       "competitiveProduct BOOLEAN", 
       "originalFinalPrice FLOAT",
       "originalMetaBriteItemPrice FLOAT", 
       "deleted VARCHAR(255)", 
       "priceAfterCoupon FLOAT",
       "metabriteCampaignId VARCHAR(255)"

    ]

    brands_columns_info = [
        "_id VARCHAR(255)",
        "barcode VARCHAR(255)",
        "category VARCHAR(255)",
        "categoryCode VARCHAR(255)",
        "name VARCHAR(255)",
        "topBrand FLOAT",
        "brandCode VARCHAR(255)",
        "cpg_ref VARCHAR(255)",
        "cpg_oid VARCHAR(255)"
        # Add more columns as needed
    ]

    # Import users.csv
    create_table_from_csv('users.csv', 'users', users_columns_info)

    # Import receipts.csv
    create_table_from_csv('receipts_og.csv', 'receipts', receipts_columns_info)

    create_table_from_csv('receipts_item.csv', 'receipts_item', receipt_items_columns_info)

    # Import brands.csv
    create_table_from_csv('brands.csv', 'brands', brands_columns_info)

    # Commit changes
    conn.commit()

except Error as e:
    print(f"Error connecting to PostgreSQL: {e}")
finally:
    if conn:
        cursor.close()
        conn.close()
        print("PostgreSQL connection is closed.")

Connected to PostgreSQL!
Table users created successfully!
Data imported into users!
Table receipts created successfully!
Data imported into receipts!
Table receipts_item created successfully!
Data imported into receipts_item!
Table brands created successfully!
Data imported into brands!
PostgreSQL connection is closed.


## Performing Exploratory Data Analysis

In [327]:
# Load the CSV files into DataFrames
users_df = pd.read_csv('users.csv')
receipts_df = pd.read_csv('receipts.csv')
receipts_item_df = pd.read_csv('receipts_item.csv')
brands_df = pd.read_csv('brands.csv')

# Display unique values in potential fields
print("Receipts - Description:")
print(receipts_item_df['description'].unique())

print("\nBrands - Name:")
print(brands_df['name'].unique())

print("\nReceipts - User IDs:")
print(receipts_df['userId'].unique())

print("\nUsers - IDs:")
print(users_df['_id'].unique())

Receipts - Description:
['ITEM NOT FOUND'
 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ' nan ...
 'spigen thin fit designed for iphone 12 mini case (2020) - black'
 'spigen tempered glass screen protector [glas.tr ez fit] designed foriphone 12 mini - sensor protection / 2 pack'
 'Berry Strawberry Conventional, 16 Ounce']

Brands - Name:
['test brand @1612366101024' 'Starbucks' 'test brand @1612366146176' ...
 'test brand @1598639215217' 'LIPTON TEA Leaves'
 'test brand @1613158231643']

Receipts - User IDs:
['5ff1e1eacfcf6c399c274ae6' '5ff1e194b6a9d73a3a9f1052'
 '5ff1e1f1cfcf6c399c274b0b' '5ff1e1e4cfcf6c399c274ac3'
 '5f9c74f7c88c1415cbddb839' '5ff1e1e9b6a9d73a3a9f10f6'
 '5ff1e1dfcfcf6c399c274ab3' '5ff1e1b4cfcf6c399c274a54'
 '5f9c74e3f1937815bd2c1d73' '5ff1e196cfcf6c399c274a38'
 '5ff370c562fde912123a5e0e' '5ff36d0362fde912123a5535'
 '5ff36be7135e7011bcb856d3' '54943462e4b07e684157a532'
 '5ff36a3862fde912123a4460' '5ff36d78135e7011bcb86488'
 '5964eb07e4b03efd0c0f267b' '5

In [329]:
receipts_item_df['description'].unique()

array(['ITEM NOT FOUND',
       'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
       nan, ...,
       'spigen thin fit designed for iphone 12 mini case (2020) - black',
       'spigen tempered glass screen protector [glas.tr ez fit] designed foriphone 12 mini - sensor protection / 2 pack',
       'Berry Strawberry Conventional, 16 Ounce'], dtype=object)

## Trying to extract/match brand_names from Receipts - Description using Regex 

    - As we were not able to find a way to join brands with either Receipts or Users table, hence we tried to match/extract the brand names from Receipts description

In [193]:


# Function to extract potential brand names from description
def extract_brand_names(description):
    brand_names = []
    words = re.findall(r'\b\w+\b', description.lower())
    for i in range(len(words)):
        for j in range(i+1, len(words)+1):
            brand_name = ' '.join(words[i:j])
            brand_names.append(brand_name)
    return brand_names

# Function to find the best match
def find_best_match(description, brands):
    best_match = ''
    best_score = 0
    
    # Check for 'ITEM NOT FOUND' or NaN in description
    if 'ITEM NOT FOUND' in str(description) or pd.isna(description):
        return ''
    
    potential_brands = extract_brand_names(description)
    
    for brand in brands:
        for potential_brand in potential_brands:
            score = fuzz.token_sort_ratio(potential_brand, brand.lower())
            if score > best_score:
                best_match = brand
                best_score = score
    
    return best_match


In [194]:
# Create a new column 'brand_name' in receipts_df
receipts_df['brand_name'] = receipts_item_df['description'].apply(lambda x: find_best_match(x, brands_df['name'].tolist()))

In [195]:
receipts_df['brand_name'].value_counts()

brand_name
                      700
Terrapin               50
Nutri-Grain            44
Huggies                28
Oscar Mayer            24
                     ... 
KLONDIKE                1
Good Seasons            1
Zima                    1
MOMOFUKU Sauce          1
Malibu® Strawberry      1
Name: count, Length: 104, dtype: int64

In [196]:
receipts_df[['description', 'brand_name']][:60]

,description,brand_name
0,ITEM NOT FOUND,
1,ITEM NOT FOUND,
2,NaN,
3,ITEM NOT FOUND,
4,ITEM NOT FOUND,
5,ITEM NOT FOUND,
6,MSSN TORTLLA,Total™
7,"Old El Paso Mild Chopped Green Chiles, 4.5 Oz",DEGREE
8,ITEM NOT FOUND,
9,ITEM NOT FOUND,


In [197]:
receipts_df.to_csv('/Users/charumangla/Downloads/fetch_assessment/receipts.csv')

In [198]:
receipts_df.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId', 'barcode',
       'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'brand_name'],
      dtype='object')

## EDA

In [330]:
users_df

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03T15:25:30.554000Z,2021-01-03T15:25:30.596999Z,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03T15:24:04.800000Z,2021-01-03T15:25:37.857999Z,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,fetch-staff,NaN,NaN


In [331]:
users_df.isnull().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

### NULLS/Missing Values in lastLogin, signUpSource, and state - in Users data

In [332]:
print(users_df.describe())

                             _id active                  createdDate  \
count                        495    495                          495   
unique                       212      2                          212   
top     54943462e4b07e684157a532   True  2014-12-19T14:21:22.381000Z   
freq                          20    494                           20   

                          lastLogin      role signUpSource state  
count                           433       495          447   439  
unique                          172         2            2     8  
top     2021-03-05T16:52:23.204000Z  consumer        Email    WI  
freq                             20       413          443   396  


In [333]:
print(users_df.duplicated().sum())

283


In [334]:
users_df['state'].value_counts()

state
WI    396
NH     20
AL     12
OH      5
IL      3
KY      1
CO      1
SC      1
Name: count, dtype: int64

In [335]:
users_df['role'].value_counts()

role
consumer       413
fetch-staff     82
Name: count, dtype: int64

In [336]:
users_df.dtypes

_id             object
active            bool
createdDate     object
lastLogin       object
role            object
signUpSource    object
state           object
dtype: object

In [342]:
## Duplicate columns

def find_duplicated_columns(df):
    duplicated_columns = []
    for column in df.columns:
        if df[column].duplicated().any():
            duplicated_columns.append(column)
    return duplicated_columns


In [343]:
find_duplicated_columns(users_df)

['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource', 'state']

### Duplicate values in columns of _id, active, createdDate, lastLogin, role, signUpSource, and state in Users data

In [344]:
find_duplicated_columns(brands_df)

['barcode',
 'category',
 'categoryCode',
 'name',
 'topBrand',
 'brandCode',
 'cpg_ref',
 'cpg_oid']

In [340]:
find_duplicated_columns(receipts_df)

['bonusPointsEarned',
 'bonusPointsEarnedReason',
 'createDate',
 'dateScanned',
 'finishedDate',
 'modifyDate',
 'pointsAwardedDate',
 'pointsEarned',
 'purchaseDate',
 'purchasedItemCount',
 'rewardsReceiptStatus',
 'totalSpent',
 'userId',
 'brand_name']

In [345]:
find_duplicated_columns(receipts_item_df)

['_id',
 'barcode',
 'description',
 'finalPrice',
 'itemPrice',
 'needsFetchReview',
 'partnerItemId',
 'preventTargetGapPoints',
 'quantityPurchased',
 'userFlaggedBarcode',
 'userFlaggedNewItem',
 'userFlaggedPrice',
 'userFlaggedQuantity',
 'needsFetchReviewReason',
 'pointsNotAwardedReason',
 'pointsPayerId',
 'rewardsGroup',
 'rewardsProductPartnerId',
 'userFlaggedDescription',
 'originalMetaBriteBarcode',
 'originalMetaBriteDescription',
 'brandCode',
 'competitorRewardsGroup',
 'discountedItemPrice',
 'originalReceiptItemText',
 'itemNumber',
 'originalMetaBriteQuantityPurchased',
 'targetPrice',
 'competitiveProduct',
 'originalFinalPrice',
 'originalMetaBriteItemPrice',
 'deleted',
 'priceAfterCoupon',
 'metabriteCampaignId']

In [346]:
brands_df.isnull().sum()

_id                0
barcode            0
category         155
categoryCode     650
name               0
topBrand         612
brandCode        269
cpg_ref         1167
cpg_oid            0
dtype: int64

### Nulls/Missing values in category, categoryCode, topBrand, brandCode, and cpg_ref - in brands data

In [347]:
receipts_df.isnull().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
brand_name                 700
dtype: int64

### Nulls/missing values in bonusPointsEarned, bonusPointsEarnedReason, finshedDate, pointsAwardedDate, pointsEarned, purchaseDate, purchasedItemCount, totalSpent - in receipts data

In [224]:
brands_df['brandCode'].value_counts()[50:100]

brandCode
CAMPO VIEJO                        1
KAHLUA                             1
MUMM NAPA                          1
MALFY                              1
TEST BRANDCODE @1610039590990      1
TEST BRANDCODE @1610495622510      1
TEST BRANDCODE @1610496646104      1
ONE                                1
FLINTSTONES MULTIVITAMIN GUMMY     1
FOSTER'S                           1
F WHITLOCK AND SONS                1
TEST BRANDCODE @1610046687007      1
TEST BRANDCODE @1610046687035      1
TEST BRANDCODE @1610046686980      1
TEST BRANDCODE @1599097538572      1
I CAN'T BELIEVE IT'S NOT BUTTER    1
TEST BRANDCODE @1607463684777      1
BAKEN-ETS                          1
LIBERTE                            1
APPLE CINNAMON CHEERIOS            1
ST STEFANUS                        1
DOLE CHILLED FRUIT JUICES          1
SUAVE                              1
CLASSICO                           1
STROEHMANN                         1
BUBLY                              1
1915 BOLTHOUSE FARMS        

In [348]:
users_df.describe()

,_id,active,createdDate,lastLogin,role,signUpSource,state
count,495,495,495,433,495,447,439
unique,212,2,212,172,2,2,8
top,54943462e4b07e684157a532,True,2014-12-19T14:21:22.381000Z,2021-03-05T16:52:23.204000Z,consumer,Email,WI
freq,20,494,20,20,413,443,396


In [349]:
receipts_df.describe()

,bonusPointsEarned,pointsEarned,purchasedItemCount,totalSpent
count,544.000000,609.000000,635.00000,684.000000
mean,238.893382,585.962890,14.75748,77.796857
std,299.091731,1357.166947,61.13424,347.110349
min,5.000000,0.000000,0.00000,0.000000
25%,5.000000,5.000000,1.00000,1.000000
50%,45.000000,150.000000,2.00000,18.200000
75%,500.000000,750.000000,5.00000,34.960000
max,750.000000,10199.800000,689.00000,4721.950000


In [350]:
brands_df.describe()

,barcode,topBrand,cpg_ref
count,1.167000e+03,555.000000,0.0
mean,5.111115e+11,0.055856,NaN
std,2.874497e+05,0.229850,NaN
min,5.111110e+11,0.000000,NaN
25%,5.111112e+11,0.000000,NaN
50%,5.111114e+11,0.000000,NaN
75%,5.111117e+11,0.000000,NaN
max,5.111119e+11,1.000000,NaN


In [351]:
users_df.dtypes

_id             object
active            bool
createdDate     object
lastLogin       object
role            object
signUpSource    object
state           object
dtype: object

In [352]:
receipts_df.dtypes

_id                         object
bonusPointsEarned          float64
bonusPointsEarnedReason     object
createDate                  object
dateScanned                 object
finishedDate                object
modifyDate                  object
pointsAwardedDate           object
pointsEarned               float64
purchaseDate                object
purchasedItemCount         float64
rewardsReceiptStatus        object
totalSpent                 float64
userId                      object
brand_name                  object
dtype: object

In [353]:
brands_df['brandCode'].value_counts()

brandCode
5.11112E+11                      29
5.11111E+11                      25
GOODNITES                         2
HUGGIES                           2
STARBUCKS                         1
                                 ..
TEST BRANDCODE @1599159969028     1
TEST BRANDCODE @1597350074404     1
SEDAL                             1
RED ROCK DELI                     1
TEST BRANDCODE @1613158231644     1
Name: count, Length: 844, dtype: int64

### Inconsistent values in brandCode - brands_df